In [1]:
import pandas as pd
 

In [2]:
df = pd.read_csv("demand_profile.csv")

In [3]:
df

,date_time,energy_kwh
0,1/1/13 0:00,0.586
1,1/1/13 1:00,0.580
2,1/1/13 2:00,0.572
3,1/1/13 3:00,0.596
4,1/1/13 4:00,0.592
...,...,...
8755,31/12/13 19:00,0.450
8756,31/12/13 20:00,0.543
8757,31/12/13 21:00,1.135
8758,31/12/13 22:00,0.729


In [4]:
df['date_time'] = pd.to_datetime(df['date_time'])

In [5]:
df.dtypes


date_time     datetime64[ns]
energy_kwh           float64
dtype: object

In [6]:
df['date_time'].dtype


dtype('<M8[ns]')

In [7]:
def apply_tarrif(kwh, hour):
    if 0<= hour<7:
        rate=12
    elif 7<= hour<17:
        rate=20
    elif 17<= hour<24:
        rate=28
    else:
        raise ValueError(f'Invalid hour:{hour}')
    return rate * kwh

In [10]:
def apply_tariff_itterrows(df):
    energy_cost_list = []
    
    for index, row in df.iterrows():
        #Get electerecity
        energy_used = row["energy_kwh"]
        hour = row["date_time"].hour
        #Append cost 
        energy_cost = apply_tarrif(energy_used, hour)
        energy_cost_list.append(energy_cost)
    df['cost_cents'] = energy_cost_list
    



In [11]:
apply_tariff_itterrows(df)

In [12]:
df.head()

,date_time,energy_kwh,cost_cents
0,2013-01-01 00:00:00,0.586,7.032
1,2013-01-01 01:00:00,0.580,6.960
2,2013-01-01 02:00:00,0.572,6.864
3,2013-01-01 03:00:00,0.596,7.152
4,2013-01-01 04:00:00,0.592,7.104


In [22]:
df.set_index(df['date_time'], inplace=True)


In [8]:
def apply_tarrif_isin(df):
    #Define hours groups
    peak_hours= df.index.hour.isin(range(17, 24))
    shoulder_hours= df.index.hour.isin(range(7, 17))
    off_peak_hours= df.index.hour.isin(range(0, 7))
    
    df.loc[peak_hours, 'cent_cost'] = df.loc[peak_hours, 'energy_kwh'] * 28
    df.loc[shoulder_hours, 'cent_cost'] = df.loc[shoulder_hours, 'energy_kwh'] * 20
    df.loc[off_peak_hours, 'cent_cost'] = df.loc[off_peak_hours, 'energy_kwh'] * 12
    

In [9]:
apply_tarrif_isin(df)

In [10]:
df

,date_time,energy_kwh,cent_cost
date_time,,,
2013-01-01 00:00:00,2013-01-01 00:00:00,0.586,7.032
2013-01-01 01:00:00,2013-01-01 01:00:00,0.580,6.960
2013-01-01 02:00:00,2013-01-01 02:00:00,0.572,6.864
2013-01-01 03:00:00,2013-01-01 03:00:00,0.596,7.152
2013-01-01 04:00:00,2013-01-01 04:00:00,0.592,7.104
...,...,...,...
2013-12-31 19:00:00,2013-12-31 19:00:00,0.450,12.600
2013-12-31 20:00:00,2013-12-31 20:00:00,0.543,15.204
2013-12-31 21:00:00,2013-12-31 21:00:00,1.135,31.780


In [12]:
#Another way

In [19]:
def apply_tarrif_cut(df):
    
    cents_per_kwh = pd.cut(x=df.index.hour, bins=[0,7,17,24], include_lowest=True, labels=[12,20,28]).astype(int)
    df['cost_cents_cut'] = cents_per_kwh * df['energy_kwh']



In [20]:
apply_tarrif_cut(df)
df

,date_time,energy_kwh,cent_cost,cost_cents_cut
date_time,,,,
2013-01-01 00:00:00,2013-01-01 00:00:00,0.586,7.032,7.032
2013-01-01 01:00:00,2013-01-01 01:00:00,0.580,6.960,6.960
2013-01-01 02:00:00,2013-01-01 02:00:00,0.572,6.864,6.864
2013-01-01 03:00:00,2013-01-01 03:00:00,0.596,7.152,7.152
2013-01-01 04:00:00,2013-01-01 04:00:00,0.592,7.104,7.104
...,...,...,...,...
2013-12-31 19:00:00,2013-12-31 19:00:00,0.450,12.600,12.600
2013-12-31 20:00:00,2013-12-31 20:00:00,0.543,15.204,15.204
2013-12-31 21:00:00,2013-12-31 21:00:00,1.135,31.780,31.780


In [23]:
#Store in HDF5 file
data_store = pd.HDFStore('processed_data.h5')

data_store['processed_data'] = df
data_store.close()

In [24]:
#Acces data store
data_store = pd.HDFStore('processed_data.h5')

processed_data_df = data_store['processed_data']
data_store.close()

In [25]:
processed_data_df

,date_time,energy_kwh,cent_cost,cost_cents_cut
date_time,,,,
2013-01-01 00:00:00,2013-01-01 00:00:00,0.586,7.032,7.032
2013-01-01 01:00:00,2013-01-01 01:00:00,0.580,6.960,6.960
2013-01-01 02:00:00,2013-01-01 02:00:00,0.572,6.864,6.864
2013-01-01 03:00:00,2013-01-01 03:00:00,0.596,7.152,7.152
2013-01-01 04:00:00,2013-01-01 04:00:00,0.592,7.104,7.104
...,...,...,...,...
2013-12-31 19:00:00,2013-12-31 19:00:00,0.450,12.600,12.600
2013-12-31 20:00:00,2013-12-31 20:00:00,0.543,15.204,15.204
2013-12-31 21:00:00,2013-12-31 21:00:00,1.135,31.780,31.780
